In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [2]:
train_df = pd.read_csv('../dataset/RoboMed/train_data.csv')
test_df = pd.read_csv('../dataset/RoboMed/test_data.csv')

In [3]:
train_df['Общее состояние'] = train_df['Общее состояние'].fillna('общее_состояние_неизвестно')
train_df['аллергия'] = train_df['аллергия'].fillna('аллергии_неизвестны')
train_df['Анамнез заболевания'] = train_df['Анамнез заболевания'].fillna('нет_анамнеза')
train_df['Внешний осмотр'] = train_df['Внешний осмотр'].fillna('нет_осмотра')

test_df['Общее состояние'] = test_df['Общее состояние'].fillna('общее_состояние_неизвестно')
test_df['аллергия'] = test_df['аллергия'].fillna('аллергии_неизвестны')
test_df['Анамнез заболевания'] = test_df['Анамнез заболевания'].fillna('нет_анамнеза')
test_df['Внешний осмотр'] = test_df['Внешний осмотр'].fillna('нет_осмотра')

In [4]:
train_size = train_df.shape[0]
print(train_size)

283086


In [156]:
# train_df = train_df.fillna('')
# test_df = test_df.fillna('')

In [7]:
train_dataframe, valid_dataframe = train_test_split(train_df, test_size=0.1,
                                                    stratify=train_df.Revisit, random_state=42)

In [8]:
def concat_features_ft(df, train=True):
    features = (
        df['Код_диагноза'] + ' | ' +
        df['Диагноз'] + ' | ' +
        'Возраст_' + df['Возраст'].astype(str) + ' | ' +
        'Пол_' + df['Пол'].astype(str) + ' | ' +
        df['Общее состояние'] + ' | ' +
        df['аллергия'] + ' | ' +
        df['Анамнез заболевания'] + ' | ' +
        df['Внешний осмотр']
    )
    
    text = features
    if train:
        text = '__label__' + df.Revisit.astype(str) + ' ' + text

    return text.str.lower().str.replace('\n', '').str.replace(', ', ' , ')

In [9]:
fasttext_train = concat_features_ft(train_dataframe)
fasttext_valid = concat_features_ft(valid_dataframe)
fasttext_test = concat_features_ft(test_df, train=False)

In [10]:
with open('fasttext_train.txt', 'w') as file:
    for line in fasttext_train:
        file.write(line + '\n')

In [11]:
with open('fasttext_valid.txt', 'w') as file:
    for line in fasttext_valid:
        file.write(line + '\n')

In [12]:
with open('fasttext_test.txt', 'w') as file:
    for line in fasttext_test:
        file.write(line + '\n')

In [14]:
! head -n 10 fasttext_test.txt

z39.2 | рутинное послеродовое наблюдение | возраст_41 | пол_2 | общее самочувствие удовлетворительное. кожные покровы и видимые слизистые обычной окраски.  язык чистый , влажный. живот мягкий ,  безболезненный во всех отделах. стул , мочеиспускание не нарушены. | аллергию на лекарства - отрицает. | нет_анамнеза | нет_осмотра
m54.2 | цервикалгия | возраст_64 | пол_2 | общее_состояние_неизвестно | не отягощен | нет_анамнеза | нет_осмотра
m41 | сколиоз | возраст_34 | пол_2 | удовлетворительное. температура тела - 36,6. сознание ясное. положение активное. пжк - развита умеренно. периферических отёков нет. кожные покровы чистые  , нормальной влажности , обычной окраски.  лимфатические узлы , доступные пальпации , не  увеличены , безболезненные. дизурические явления отрицает.  стул - без особенностей ( со слов пациентки). | аллергии_неизвестны | нет_анамнеза | нет_осмотра
j31.2 | хронический фарингит | возраст_58 | пол_2 | общее_состояние_неизвестно | аллергии_неизвестны | нет_анамнеза | общ

In [164]:
! wc fasttext_valid.txt

   28309  1122765 14445049 fasttext_valid.txt


In [25]:
! cat fasttext_train.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_train_preprocessed.txt

/bin/sh: 1: fasttext: not found


In [ ]:
!  cat fasttext_valid.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_valid_preprocessed.txt

In [67]:
!  cat fasttext_test.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_test_preprocessed.txt

In [61]:
! fasttext supervised -input fasttext_train.txt -wordNgrams 2 -loss hs -thread 32 -output model_categories -dim 300 #-pretrainedVectors ft_native_300_ru_wiki_lenta_lower_case.vec

/bin/sh: 1: fasttext: not found


In [ ]:
! fasttext test model_categories.bin fasttext_valid.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_test.txt > y_pred_test.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_valid.txt > y_pred_valid.txt

In [15]:
def read_pred(filename):
    with open(filename) as file:
        lines = file.readlines()
        y_pred = np.array([float(line.split()[1]) if line.split()[0] == '__label__1' else 1 - float(line.split()[1])
                           for line in lines])
    return y_pred

In [16]:
y_pred_valid = read_pred('y_pred_valid.txt')

In [17]:
roc_auc_score(valid_dataframe.Revisit, y_pred_valid)

0.6995795975065916

In [18]:
y_pred_test = read_pred('y_pred_test.txt')

In [19]:
submission = pd.read_csv('/root/dataset/RoboMed/sample_submission.csv')

In [20]:
submission.proba = y_pred_test

In [21]:
submission.to_csv('submission_ft.csv', index=False)